In [159]:
# importations
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
import datetime as dt
from datetime import datetime
from datetime import timedelta

In [160]:
# import datafiles
csv_airports = "../ETL_Project/CSV_files/airports.csv"
airports_df = pd.read_csv(csv_airports)
csv_flights = "../ETL_Project/CSV_files/flights.csv"
flights_df = pd.read_csv(csv_flights)

In [161]:
# drop columns with NaN and irrelevant columns
flights_df = flights_df.dropna(axis='columns')
flights_df = flights_df.drop(columns=["CANCELLED", "DIVERTED"])
flights_df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DISTANCE,SCHEDULED_ARRIVAL
0,2015,1,1,4,AS,98,ANC,SEA,5,1448,430
1,2015,1,1,4,AA,2336,LAX,PBI,10,2330,750
2,2015,1,1,4,US,840,SFO,CLT,20,2296,806
3,2015,1,1,4,AA,258,LAX,MIA,20,2342,805
4,2015,1,1,4,AS,135,SEA,ANC,25,1448,320


In [162]:
# we decided to focus on Delta flights leaving Atlanta
delta_flights = flights_df.loc[(flights_df["AIRLINE"] == "DL")]
delta_flights = delta_flights.loc[(delta_flights["ORIGIN_AIRPORT"] == "ATL")]
delta_flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DISTANCE,SCHEDULED_ARRIVAL
729,2015,1,1,4,DL,886,ATL,LGA,645,762,853
815,2015,1,1,4,DL,1818,ATL,MCO,655,404,822
1167,2015,1,1,4,DL,2499,ATL,SLC,720,1590,940
1215,2015,1,1,4,DL,1260,ATL,BDL,725,859,940
1273,2015,1,1,4,DL,2103,ATL,MIA,730,594,926


In [163]:
# dropped more unnecessary columns and renamed a column
airports_df = airports_df.drop(columns=["COUNTRY", "LATITUDE", "LONGITUDE"])
airports_df = airports_df.rename(columns={"IATA_CODE" : "DESTINATION_AIRPORT"})
airports_df.head()

,DESTINATION_AIRPORT,AIRPORT,CITY,STATE
0,ABE,Lehigh Valley International Airport,Allentown,PA
1,ABI,Abilene Regional Airport,Abilene,TX
2,ABQ,Albuquerque International Sunport,Albuquerque,NM
3,ABR,Aberdeen Regional Airport,Aberdeen,SD
4,ABY,Southwest Georgia Regional Airport,Albany,GA


In [164]:
# merged two dataframes together
combined_data_df = pd.merge(airports_df, delta_flights,
                                 how='outer', on='DESTINATION_AIRPORT')
combined_data_df = combined_data_df.dropna(axis='rows')
combined_data_df

,DESTINATION_AIRPORT,AIRPORT,CITY,STATE,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,SCHEDULED_DEPARTURE,DISTANCE,SCHEDULED_ARRIVAL
0,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,4.0,7.0,DL,2036.0,ATL,2101.0,692.0,2251.0
1,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,5.0,1.0,DL,1411.0,ATL,2034.0,692.0,2225.0
2,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,6.0,2.0,DL,1411.0,ATL,2034.0,692.0,2225.0
3,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,7.0,3.0,DL,1411.0,ATL,2034.0,692.0,2225.0
4,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,8.0,4.0,DL,1411.0,ATL,2034.0,692.0,2225.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42381,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,2015.0,3.0,5.0,4.0,DL,2119.0,ATL,2115.0,589.0,2214.0
42382,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,2015.0,3.0,6.0,5.0,DL,2119.0,ATL,2115.0,589.0,2214.0
42383,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,2015.0,3.0,7.0,6.0,DL,2119.0,ATL,2115.0,589.0,2212.0
42384,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,2015.0,3.0,8.0,7.0,DL,2119.0,ATL,2115.0,589.0,2214.0


In [165]:
# changed weekday from number to name 
dayOfWeek={1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday', 7:'Sunday'}
combined_data_df['WEEKDAY'] = combined_data_df['DAY_OF_WEEK'].map(dayOfWeek).rename()
combined_data_df.head()

,DESTINATION_AIRPORT,AIRPORT,CITY,STATE,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,ORIGIN_AIRPORT,SCHEDULED_DEPARTURE,DISTANCE,SCHEDULED_ARRIVAL,WEEKDAY
0,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,4.0,7.0,DL,2036.0,ATL,2101.0,692.0,2251.0,Sunday
1,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,5.0,1.0,DL,1411.0,ATL,2034.0,692.0,2225.0,Monday
2,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,6.0,2.0,DL,1411.0,ATL,2034.0,692.0,2225.0,Tuesday
3,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,7.0,3.0,DL,1411.0,ATL,2034.0,692.0,2225.0,Wednesday
4,ABE,Lehigh Valley International Airport,Allentown,PA,2015.0,1.0,8.0,4.0,DL,1411.0,ATL,2034.0,692.0,2225.0,Thursday


In [166]:
# dropped original number of weekday column
combined_data_df = combined_data_df.drop(columns=["DAY_OF_WEEK"])

In [167]:
# rearranged column order and formatted numbers
combined_data_df = combined_data_df[["MONTH", "DAY" , "YEAR", "WEEKDAY", "DESTINATION_AIRPORT", "AIRPORT", "CITY", "STATE", "ORIGIN_AIRPORT", "AIRLINE", "FLIGHT_NUMBER", "SCHEDULED_DEPARTURE", "SCHEDULED_ARRIVAL", "DISTANCE"]]
combined_data_df["MONTH"] = combined_data_df["MONTH"].astype(float).map("{:.0f}".format)
combined_data_df["DAY"] = combined_data_df["DAY"].astype(float).map("{:.0f}".format)
combined_data_df["YEAR"] = combined_data_df["YEAR"].astype(float).map("{:.0f}".format)
combined_data_df["FLIGHT_NUMBER"] = combined_data_df["FLIGHT_NUMBER"].astype(float).map("{:.0f}".format)
combined_data_df["SCHEDULED_DEPARTURE"] = combined_data_df["SCHEDULED_DEPARTURE"].astype(float).map("{:.0f}".format)
combined_data_df["SCHEDULED_ARRIVAL"]= combined_data_df["SCHEDULED_ARRIVAL"].astype(float).map("{:.0f}".format)
combined_data_df.head()

,MONTH,DAY,YEAR,WEEKDAY,DESTINATION_AIRPORT,AIRPORT,CITY,STATE,ORIGIN_AIRPORT,AIRLINE,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,SCHEDULED_ARRIVAL,DISTANCE
0,1,4,2015,Sunday,ABE,Lehigh Valley International Airport,Allentown,PA,ATL,DL,2036,2101,2251,692.0
1,1,5,2015,Monday,ABE,Lehigh Valley International Airport,Allentown,PA,ATL,DL,1411,2034,2225,692.0
2,1,6,2015,Tuesday,ABE,Lehigh Valley International Airport,Allentown,PA,ATL,DL,1411,2034,2225,692.0
3,1,7,2015,Wednesday,ABE,Lehigh Valley International Airport,Allentown,PA,ATL,DL,1411,2034,2225,692.0
4,1,8,2015,Thursday,ABE,Lehigh Valley International Airport,Allentown,PA,ATL,DL,1411,2034,2225,692.0


In [168]:
# set column names to lowercase to be able to interact with postgres
combined_data_df = combined_data_df.rename(columns={"MONTH":"month","DAY":"day","YEAR": "year","WEEKDAY":"weekday","DESTINATION_AIRPORT":"destination_airport","AIRPORT":"airport", "CITY":"city", "STATE": "state", "ORIGIN_AIRPORT": "origin_airport", "AIRLINE":"airline", "FLIGHT_NUMBER":"flight_number", "SCHEDULED_DEPARTURE":"scheduled_departure", "SCHEDULED_ARRIVAL":"scheduled_arrival", "DISTANCE":"distance"})

In [169]:
# set up engine
rds_connection_string = "postgres:Oscarjoy0305!@localhost:5432/ETL_PROJECT"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [170]:
# referenced postgres table names
engine.table_names()

['combined_data_df']

In [171]:
# connected dataframe to postgres database
combined_data_df.to_sql(name='combined_data_df', con=engine, if_exists='append', index=False)